In [1]:
import sys
import math

from src.raytracer.raytracer import point3, vec3, color, unit
from src.raytracer.raytracer import unit, dot, random_in_unit_sphere
from src.raytracer.ray import ray
from src.raytracer import hittables
from src.raytracer import rweekend
from src.raytracer import camera
from src.raytracer import writeimg

In [2]:
def ray_color(r, world, depth):
    rec = hittables.hit_record()
    if depth <= 0:
        return color(0, 0, 0)
    
    hit_out = world.hit(r, 0.0, rweekend.infinity, rec)
    if hit_out[0]:
        rec = hit_out[1]
        target = rec.p + rec.normal + random_in_unit_sphere()
        out = 0.5 * ray_color(ray(rec.p, target - rec.p), world, depth-1)
        return out
    
    unit_direction = unit(r.direction)
    t = 0.5 * (unit_direction.y + 1.0)
    return (1.0 - t) * color(1.0, 1.0, 1.0) \
            + t * color(0.5, 0.7, 1.0)

In [3]:
##

In [4]:
# image params
aspect_ratio = 16.0 / 9.0
image_width = 400
image_height = math.floor(image_width / aspect_ratio)
samples_per_pixel = 25 # spp 25, md 10, 30 min
max_depth = 10

In [5]:
# world
world = hittables.hittable_list()
world.add(hittables.sphere(point3(0, 0, -1), 0.5))
world.add(hittables.sphere(point3(0, -100.5, -1), 100.0))

In [6]:
# camera params
cam = camera.camera()

In [7]:
# render image
outimg = writeimg.writeppm(image_width, image_height,
                           'outfile.ppm', 'P3', 255)
outimg.write_head()
for j in range(image_height-1, -1, -1):
    sys.stdout.write("\r%d%%" % j)
    sys.stdout.flush()
    for i in range(0, image_width):
        pixel_color = color(0, 0, 0)
        for s in range(0, samples_per_pixel):
            u = float(i + rweekend.random_double())/(image_width - 1)
            v = float(j + rweekend.random_double())/(image_height - 1)
            r = cam.get_ray(u, v)
            pixel_color += ray_color(r, world, max_depth)
            #print("pixel_color", pixel_color)
        outimg.write_color(pixel_color, samples_per_pixel)
sys.stdout.write("done")

0%%%done

In [8]:
outimg.check_valid()

[True, 'params OK']

In [9]:
outimg.write_color_file()